In [2]:
import pandas as pd
import numpy as np

import pickle

# gensim
from gensim import corpora, models, similarities, matutils

# sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import nltk
import spacy


Loading in game text data:

In [3]:
with open('/Users/robertpagano/metis_data/project_4/text_dataframes/games_text_2016_2019.pickle', 'rb') as f:
    df = pickle.load(f)

In [4]:
df.head()

,final_state,id,name,blurb,category_name
1796,successful,269726791,Emanations Playing Cards | A Study of Real Magic,An artistic professional pack of playing cards...,Playing Cards
2575,successful,1697621882,FAT STACKS: The Pancake Stacking Strategy Card...,Make it rain pancakes in FAT STACKS. \nCreate ...,Tabletop Games
2594,successful,2046938895,Royal Heroz - Fantasy Strategy Playing Cards,"Endless playability, stunning artwork and fant...",Playing Cards
2665,successful,1642293087,Coral Islands : 2 interactive dice-stacking ga...,"In 'Coral', use dice to make shapes in 3D! In ...",Tabletop Games
2668,failed,932587626,3D Shopping with WalkTheWeb 3D Technology!,Online 3D Stores for businesses that blur 3D G...,Live Games


In [5]:
df.shape

(5902, 5)

In [6]:
X_blurb = df['blurb']

In [7]:
X_blurb.head()

1796    An artistic professional pack of playing cards...
2575    Make it rain pancakes in FAT STACKS. \nCreate ...
2594    Endless playability, stunning artwork and fant...
2665    In 'Coral', use dice to make shapes in 3D! In ...
2668    Online 3D Stores for businesses that blur 3D G...
Name: blurb, dtype: object

BACK UP. What I need to do is create functions or something to do my text preprocessing on the column ITSELF.

And then I load it into the vecotorizer

In [25]:
!python -m spacy download en

    100% |████████████████████████████████| 37.4MB 211kB/s ta 0:00:011    86% |███████████████████████████▌    | 32.1MB 4.9MB/s eta 0:00:02
  Running setup.py install for en-core-web-sm ... done

    Linking successful
    /anaconda3/lib/python3.6/site-packages/en_core_web_sm -->
    /anaconda3/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [8]:
from nltk.corpus import stopwords

In [9]:
import re
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

In [10]:
df['blurb'].head()
df.head()

,final_state,id,name,blurb,category_name
1796,successful,269726791,Emanations Playing Cards | A Study of Real Magic,An artistic professional pack of playing cards...,Playing Cards
2575,successful,1697621882,FAT STACKS: The Pancake Stacking Strategy Card...,Make it rain pancakes in FAT STACKS. \nCreate ...,Tabletop Games
2594,successful,2046938895,Royal Heroz - Fantasy Strategy Playing Cards,"Endless playability, stunning artwork and fant...",Playing Cards
2665,successful,1642293087,Coral Islands : 2 interactive dice-stacking ga...,"In 'Coral', use dice to make shapes in 3D! In ...",Tabletop Games
2668,failed,932587626,3D Shopping with WalkTheWeb 3D Technology!,Online 3D Stores for businesses that blur 3D G...,Live Games


In [11]:
df['blurb_processed'] = df['blurb'].apply(lambda x:pre_process(x))

In [12]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [13]:
from textblob import TextBlob, Word

In [14]:
def pre_process2(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    #lemmatize with Spacy
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc])
    
    return text

In [15]:
df['blurb_processed_2'] = df['blurb'].apply(lambda x:pre_process2(x))

In [16]:
df.head()

,final_state,id,name,blurb,category_name,blurb_processed,blurb_processed_2
1796,successful,269726791,Emanations Playing Cards | A Study of Real Magic,An artistic professional pack of playing cards...,Playing Cards,an artistic professional pack of playing cards...,an artistic professional pack of playing card ...
2575,successful,1697621882,FAT STACKS: The Pancake Stacking Strategy Card...,Make it rain pancakes in FAT STACKS. \nCreate ...,Tabletop Games,make it rain pancakes in fat stacks create the...,make -PRON- rain pancake in fat stack create t...
2594,successful,2046938895,Royal Heroz - Fantasy Strategy Playing Cards,"Endless playability, stunning artwork and fant...",Playing Cards,endless playability stunning artwork and fanta...,endless playability stunning artwork and fanta...
2665,successful,1642293087,Coral Islands : 2 interactive dice-stacking ga...,"In 'Coral', use dice to make shapes in 3D! In ...",Tabletop Games,in coral use dice to make shapes in d in islan...,in coral use dice to make shape in d in island...
2668,failed,932587626,3D Shopping with WalkTheWeb 3D Technology!,Online 3D Stores for businesses that blur 3D G...,Live Games,online d stores for businesses that blur d gam...,online d store for business that blur d game w...


In [17]:
X_blurb_2 = df['blurb_processed_2']

In [18]:
tfidf = TfidfVectorizer(stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")


In [19]:
tfidf2 = TfidfVectorizer(stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")


In [20]:
tfidf3 = TfidfVectorizer(stop_words='english', token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))


In [21]:
bag_of_words = tfidf2.fit_transform(X_blurb)

In [22]:
feature_names = tfidf2.get_feature_names()
pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

,aaa,aapo,aargh,ab,aba,abandonado,abandoned,abandonned,abbey,abc,...,zum,zur,zusammen,zweck,zwergen,zwiercies,zx,zy,zyde,zynvaded
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Below I am trying the tfidf transform again, after preprocessing text

In [23]:
bag_of_words_2 = tfidf.fit_transform(X_blurb_2)

In [24]:
bag_of_words_2.shape

(5902, 10022)

In [25]:
feature_names_2 = tfidf.get_feature_names()
pd.DataFrame(bag_of_words_2.toarray(), columns = feature_names_2)

,aaa,aapo,aargh,ab,aba,abandon,abandonado,abandonned,abbey,abc,...,zur,zusamman,zusammen,zweck,zwergen,zwiercie,zx,zy,zyde,zynvad
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
# Acronynms: Latent Semantic Analysis (LSA) is just another name for 
#  Signular Value Decomposition (SVD) applied to Natural Language Processing (NLP)
lsa = TruncatedSVD(4)
doc_topic = lsa.fit_transform(bag_of_words_2)
lsa.explained_variance_ratio_

array([0.00569487, 0.00868325, 0.00592887, 0.0045029 ])

In [28]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [29]:
display_topics(lsa, tfidf.get_feature_names(), 5)


Topic  0
pron, game, card, play, fun

Topic  1
card, playing, deck, design, print

Topic  2
pron, deck, playing, card, print

Topic  3
world, rpg, adventure, playing, fantasy


Ok so this is not working well yet...

In [30]:
# Acronynms: Latent Semantic Analysis (LSA) is just another name for 
#  Signular Value Decomposition (SVD) applied to Natural Language Processing (NLP)
lsa2 = TruncatedSVD(2)
doc_topic = lsa2.fit_transform(bag_of_words)
lsa2.explained_variance_ratio_

array([0.0035062, 0.0074047])

In [31]:
display_topics(lsa2, tfidf2.get_feature_names(), 5)


Topic  0
game, card, cards, playing, deck

Topic  1
cards, playing, deck, printed, custom


Going to try again with ngrams:

In [32]:
bag_of_words_3 = tfidf3.fit_transform(X_blurb_2)

In [33]:
bag_of_words_3.shape

(5902, 56032)

In [34]:
lsa3 = TruncatedSVD(4)
doc_topic = lsa3.fit_transform(bag_of_words_3)
lsa3.explained_variance_ratio_

array([0.00225564, 0.00421463, 0.00280057, 0.00206042])

In [35]:
display_topics(lsa3, tfidf3.get_feature_names(), 5)


Topic  0
pron, game, card, card game, play

Topic  1
card, playing card, playing, deck, print

Topic  2
game, card game, player, game player, fun

Topic  3
card game, card, pron, fun, play card


In [36]:
tfidf4 = TfidfVectorizer(stop_words='english', token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2), sublinear_tf=True)


In [37]:
bag_of_words_4 = tfidf4.fit_transform(X_blurb_2)

In [38]:
bag_of_words_3.shape

(5902, 56032)

In [39]:
lsa4 = TruncatedSVD(20)
doc_topic = lsa4.fit_transform(bag_of_words_4)
lsa4.explained_variance_ratio_

array([0.00174973, 0.00374814, 0.00240708, 0.00200577, 0.00174532,
       0.00153183, 0.00147784, 0.00140342, 0.00137219, 0.0012976 ,
       0.00126706, 0.0012534 , 0.0012246 , 0.00121178, 0.00116889,
       0.00114738, 0.00113098, 0.00110763, 0.00108639, 0.0010621 ])

In [40]:
display_topics(lsa4, tfidf4.get_feature_names(), 20)


Topic  0
pron, game, card, card game, play, playing, player, deck, playing card, fun, game pron, new, world, make, friend, create, design, base, game player, pron pron

Topic  1
card, playing card, playing, deck, print, design, custom, deck playing, uspcc, play card, print uspcc, card deck, card inspire, magician, card print, custom deck, inspire, luxury, poker, card design

Topic  2
card game, game, card, player, game player, fun, strategy, fast, play, age, board, board game, fun card, strategic, family, minute, play card, fast paced, paced, base

Topic  3
adventure, world, rpg, game, fantasy, action, base, playing, role, set, role playing, playing game, story, adventure game, mobile, player, new, tabletop, open world, open

Topic  4
play, play card, deck, deck play, pron play, inspire, rpg, game play, play pron, play game, card inspire, world, unique, mobile, free, adventure, deck card, custom deck, easy, magician

Topic  5
world, card game, novel, visual, visual novel, card, fantas

do a plot of inertia, see where it levels off with # of components, and then run kmeans clustering on those components to come up with clusters for projects

In [41]:
type(doc_topic)

numpy.ndarray

In [42]:
lsa4_df = pd.DataFrame(data=doc_topic)

In [43]:
lsa4_df.shape

(5902, 20)

In [44]:
lsa4_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.118766,0.182460,-0.037602,0.038939,-0.048892,-0.046611,0.001251,-0.006294,0.010233,-0.005970,-0.006676,-0.004561,0.023085,0.029665,-0.003810,0.003784,-0.000054,0.000972,-0.012741,0.026417
1,0.052764,-0.033607,-0.045281,-0.032657,-0.013082,-0.014322,0.010936,0.002551,-0.029429,0.026472,0.021774,0.010629,0.055882,-0.027455,0.014829,-0.019903,0.028357,0.008947,0.025654,0.052767
2,0.017983,0.001342,-0.003237,0.031355,0.005765,0.025151,0.009364,-0.000253,0.003074,0.012427,0.007717,-0.012621,-0.008298,-0.014354,-0.013619,-0.012021,0.005546,0.003372,0.021002,-0.008529
3,0.035802,-0.011526,-0.010439,0.004968,-0.004683,-0.007059,0.004393,0.018622,-0.012417,0.026625,-0.017173,0.019009,0.046798,-0.012569,-0.012555,-0.057746,0.018758,-0.012114,0.045525,0.010689
4,0.085047,-0.055662,-0.040012,0.035371,0.012490,-0.002498,-0.031839,0.000353,-0.022461,-0.033537,0.030959,-0.055708,-0.033630,0.028152,-0.002298,0.037401,0.035000,0.018628,0.008067,-0.015034


In [45]:
lsa4_df = lsa4_df.add_prefix('topic_')

In [46]:
lsa4_df.head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,0.118766,0.182460,-0.037602,0.038939,-0.048892,-0.046611,0.001251,-0.006294,0.010233,-0.005970,-0.006676,-0.004561,0.023085,0.029665,-0.003810,0.003784,-0.000054,0.000972,-0.012741,0.026417
1,0.052764,-0.033607,-0.045281,-0.032657,-0.013082,-0.014322,0.010936,0.002551,-0.029429,0.026472,0.021774,0.010629,0.055882,-0.027455,0.014829,-0.019903,0.028357,0.008947,0.025654,0.052767
2,0.017983,0.001342,-0.003237,0.031355,0.005765,0.025151,0.009364,-0.000253,0.003074,0.012427,0.007717,-0.012621,-0.008298,-0.014354,-0.013619,-0.012021,0.005546,0.003372,0.021002,-0.008529
3,0.035802,-0.011526,-0.010439,0.004968,-0.004683,-0.007059,0.004393,0.018622,-0.012417,0.026625,-0.017173,0.019009,0.046798,-0.012569,-0.012555,-0.057746,0.018758,-0.012114,0.045525,0.010689
4,0.085047,-0.055662,-0.040012,0.035371,0.012490,-0.002498,-0.031839,0.000353,-0.022461,-0.033537,0.030959,-0.055708,-0.033630,0.028152,-0.002298,0.037401,0.035000,0.018628,0.008067,-0.015034


In [47]:
df_topics = df.reset_index()
df_topics.head()

,index,final_state,id,name,blurb,category_name,blurb_processed,blurb_processed_2
0,1796,successful,269726791,Emanations Playing Cards | A Study of Real Magic,An artistic professional pack of playing cards...,Playing Cards,an artistic professional pack of playing cards...,an artistic professional pack of playing card ...
1,2575,successful,1697621882,FAT STACKS: The Pancake Stacking Strategy Card...,Make it rain pancakes in FAT STACKS. \nCreate ...,Tabletop Games,make it rain pancakes in fat stacks create the...,make -PRON- rain pancake in fat stack create t...
2,2594,successful,2046938895,Royal Heroz - Fantasy Strategy Playing Cards,"Endless playability, stunning artwork and fant...",Playing Cards,endless playability stunning artwork and fanta...,endless playability stunning artwork and fanta...
3,2665,successful,1642293087,Coral Islands : 2 interactive dice-stacking ga...,"In 'Coral', use dice to make shapes in 3D! In ...",Tabletop Games,in coral use dice to make shapes in d in islan...,in coral use dice to make shape in d in island...
4,2668,failed,932587626,3D Shopping with WalkTheWeb 3D Technology!,Online 3D Stores for businesses that blur 3D G...,Live Games,online d stores for businesses that blur d gam...,online d store for business that blur d game w...


In [1]:
a

NameError: name 'a' is not defined

In [48]:
df_topics = pd.merge(df_topics, lsa4_df, left_index=True, right_index=True)

In [49]:
df_topics.shape

(5902, 28)

In [50]:
df_topics.head()

,index,final_state,id,name,blurb,category_name,blurb_processed,blurb_processed_2,topic_0,topic_1,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,1796,successful,269726791,Emanations Playing Cards | A Study of Real Magic,An artistic professional pack of playing cards...,Playing Cards,an artistic professional pack of playing cards...,an artistic professional pack of playing card ...,0.118766,0.182460,...,-0.006676,-0.004561,0.023085,0.029665,-0.003810,0.003784,-0.000054,0.000972,-0.012741,0.026417
1,2575,successful,1697621882,FAT STACKS: The Pancake Stacking Strategy Card...,Make it rain pancakes in FAT STACKS. \nCreate ...,Tabletop Games,make it rain pancakes in fat stacks create the...,make -PRON- rain pancake in fat stack create t...,0.052764,-0.033607,...,0.021774,0.010629,0.055882,-0.027455,0.014829,-0.019903,0.028357,0.008947,0.025654,0.052767
2,2594,successful,2046938895,Royal Heroz - Fantasy Strategy Playing Cards,"Endless playability, stunning artwork and fant...",Playing Cards,endless playability stunning artwork and fanta...,endless playability stunning artwork and fanta...,0.017983,0.001342,...,0.007717,-0.012621,-0.008298,-0.014354,-0.013619,-0.012021,0.005546,0.003372,0.021002,-0.008529
3,2665,successful,1642293087,Coral Islands : 2 interactive dice-stacking ga...,"In 'Coral', use dice to make shapes in 3D! In ...",Tabletop Games,in coral use dice to make shapes in d in islan...,in coral use dice to make shape in d in island...,0.035802,-0.011526,...,-0.017173,0.019009,0.046798,-0.012569,-0.012555,-0.057746,0.018758,-0.012114,0.045525,0.010689
4,2668,failed,932587626,3D Shopping with WalkTheWeb 3D Technology!,Online 3D Stores for businesses that blur 3D G...,Live Games,online d stores for businesses that blur d gam...,online d store for business that blur d game w...,0.085047,-0.055662,...,0.030959,-0.055708,-0.033630,0.028152,-0.002298,0.037401,0.035000,0.018628,0.008067,-0.015034


In [114]:
with open('/Users/robertpagano/metis_data/project_4/text_dataframes/games_and_lsatopics.pickle', 'wb') as to_write:
    pickle.dump(df_topics, to_write)

IN the interest of getting to my MVP, I'm going to move onto modeling with these 20 topics. 

MVP Steps:

 - Create models - RF and Logistic
     - Setup structure for this - i.e. cross val, metrics, etc.
     - Create models with and without LSA components (and maybe ONLY LSA components)
     - Create baseline using %rate of $ / %rate needed > 1, see how it compares to both

Future Steps:
 - add specific stop words (game, etc) and stemming
 - NMF
 - LDA
 - Look into Inertia scores for # of topics?
 - Further EDA for model (Consider adding more categories)
 - Go back to modelling. If haven't found reason for # of topics, CV on this
 - Consider adding more categories
 - Presentation Slides
 - Clustering on topics (if not LDA)
 - Recommender

Below, going to try out NMF using same bag of words I landed on with LSA, and see what comes through:

Creating models on 10, 15, 20, and 25 topics. Will do the same with LSA, and save dataframes for all 8 versions, and then cluster on each. Whichever seems to make the clusters that make most sense, I'll pick

First, creating a DF with just ID. I'll append each of the topic model DFs to this, and then run clustering on that

In [85]:
df_topic_base = df.reset_index()
df_topic_base.head()

,index,final_state,id,name,blurb,category_name,blurb_processed,blurb_processed_2
0,1796,successful,269726791,Emanations Playing Cards | A Study of Real Magic,An artistic professional pack of playing cards...,Playing Cards,an artistic professional pack of playing cards...,an artistic professional pack of playing card ...
1,2575,successful,1697621882,FAT STACKS: The Pancake Stacking Strategy Card...,Make it rain pancakes in FAT STACKS. \nCreate ...,Tabletop Games,make it rain pancakes in fat stacks create the...,make -PRON- rain pancake in fat stack create t...
2,2594,successful,2046938895,Royal Heroz - Fantasy Strategy Playing Cards,"Endless playability, stunning artwork and fant...",Playing Cards,endless playability stunning artwork and fanta...,endless playability stunning artwork and fanta...
3,2665,successful,1642293087,Coral Islands : 2 interactive dice-stacking ga...,"In 'Coral', use dice to make shapes in 3D! In ...",Tabletop Games,in coral use dice to make shapes in d in islan...,in coral use dice to make shape in d in island...
4,2668,failed,932587626,3D Shopping with WalkTheWeb 3D Technology!,Online 3D Stores for businesses that blur 3D G...,Live Games,online d stores for businesses that blur d gam...,online d store for business that blur d game w...


In [86]:
df_topic_base = df_topic_base.drop(columns=['index', 'final_state', 'name', 'blurb', 'category_name', 'blurb_processed', 'blurb_processed_2'])

In [88]:
df_topic_base.shape

(5902, 1)

NMF Models:

In [66]:
nmf_model_10 = NMF(10)
doc_topic_NMF_10 = nmf_model_10.fit_transform(bag_of_words_4)

In [74]:
nmf_model_10_df = pd.DataFrame(data=doc_topic_NMF_10)
nmf_model_10_df.shape

(5902, 10)

In [90]:
df_nmf_model_10 = pd.merge(df_topic_base, nmf_model_10_df, left_index=True, right_index=True)
df_nmf_model_10.head()

,id,0,1,2,3,4,5,6,7,8,9
0,269726791,0.000000,0.113225,0.004469,0.012575,0.000000,0.000000,0.000000,0.005337,0.005931,0.005011
1,1697621882,0.025402,0.000294,0.000000,0.001866,0.000000,0.000000,0.000000,0.000000,0.000000,0.004661
2,2046938895,0.000056,0.001136,0.001679,0.000297,0.000938,0.009968,0.009566,0.001023,0.008842,0.000000
3,1642293087,0.007226,0.003866,0.000000,0.001354,0.000000,0.001855,0.001736,0.010914,0.000000,0.013104
4,932587626,0.017310,0.000000,0.000000,0.035607,0.000000,0.022068,0.000000,0.001039,0.000000,0.000000


In [67]:
nmf_model_15 = NMF(15)
doc_topic_NMF_15 = nmf_model_15.fit_transform(bag_of_words_4)

In [75]:
nmf_model_15_df = pd.DataFrame(data=doc_topic_NMF_15)
nmf_model_15_df.shape

(5902, 15)

In [91]:
df_nmf_model_15 = pd.merge(df_topic_base, nmf_model_15_df, left_index=True, right_index=True)
df_nmf_model_15.head()

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,269726791,0.000000,0.097718,0.005496,0.000982,0.00000,0.000000,0.000000,0.004225,0.005869,0.009789,0.003608,0.001300,0.007397,0.037609,0.002789
1,1697621882,0.022888,0.000000,0.000000,0.000000,0.00000,0.000839,0.000000,0.000000,0.000000,0.000000,0.004250,0.000000,0.005223,0.001933,0.000000
2,2046938895,0.000273,0.002290,0.001052,0.000944,0.00074,0.008825,0.009738,0.001078,0.008964,0.001737,0.000000,0.000000,0.000000,0.000000,0.000000
3,1642293087,0.007297,0.000000,0.000000,0.002962,0.00000,0.001632,0.001689,0.009668,0.000283,0.001214,0.010326,0.000000,0.000000,0.012173,0.000000
4,932587626,0.014870,0.000000,0.000541,0.000638,0.00000,0.031778,0.000000,0.001021,0.000371,0.038281,0.000000,0.000513,0.006357,0.000000,0.000000


In [68]:
nmf_model_20 = NMF(20)
doc_topic_NMF_20 = nmf_model_20.fit_transform(bag_of_words_4)

In [76]:
nmf_model_20_df = pd.DataFrame(data=doc_topic_NMF_20)
nmf_model_20_df.shape

(5902, 20)

In [92]:
df_nmf_model_20 = pd.merge(df_topic_base, nmf_model_20_df, left_index=True, right_index=True)
df_nmf_model_20.head()

,id,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,269726791,0.000000,0.095625,0.005516,0.000837,0.000000,0.000000,0.000000,0.003886,0.006099,...,0.002900,0.002937,0.032018,0.006936,0.00233,0.000288,0.001805,0.000000,0.000000,0.000000
1,1697621882,0.018904,0.000000,0.000000,0.000000,0.000000,0.000738,0.000000,0.000000,0.000000,...,0.003070,0.000000,0.001610,0.004705,0.00000,0.001381,0.000000,0.000000,0.000000,0.000000
2,2046938895,0.000212,0.002750,0.000000,0.021505,0.000572,0.000351,0.005357,0.000000,0.001829,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000890,0.000000
3,1642293087,0.005297,0.000000,0.000000,0.013869,0.000000,0.000235,0.000625,0.006004,0.000000,...,0.008294,0.000000,0.009062,0.000000,0.00000,0.000000,0.000000,0.001032,0.000000,0.000000
4,932587626,0.010747,0.000000,0.001803,0.000000,0.000000,0.037434,0.000000,0.001863,0.002607,...,0.000000,0.000915,0.000000,0.008185,0.00000,0.000000,0.002128,0.012788,0.000216,0.001269


In [69]:
nmf_model_25 = NMF(25)
doc_topic_NMF_25 = nmf_model_25.fit_transform(bag_of_words_4)

In [77]:
nmf_model_25_df = pd.DataFrame(data=doc_topic_NMF_25)
nmf_model_25_df.shape

(5902, 25)

In [93]:
df_nmf_model_25 = pd.merge(df_topic_base, nmf_model_25_df, left_index=True, right_index=True)
df_nmf_model_25.head()

,id,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,269726791,0.000000,0.114443,0.004754,0.000786,0.003525,0.000000,0.000000,0.003227,0.000000,...,0.000553,0.000000,0.003372,0.000378,0.007045,0.003880,0.000000,0.000000,0.000000,0.00000
1,1697621882,0.006988,0.000000,0.000000,0.000000,0.000000,0.000864,0.000000,0.000000,0.000000,...,0.000633,0.000000,0.000000,0.000000,0.002290,0.000000,0.000000,0.000000,0.000000,0.00000
2,2046938895,0.000318,0.001595,0.000000,0.000000,0.000484,0.002506,0.005548,0.000000,0.001974,...,0.000000,0.002570,0.000000,0.003012,0.000000,0.033294,0.000000,0.000000,0.000000,0.00024
3,1642293087,0.000000,0.000000,0.000000,0.000000,0.000000,0.000974,0.000846,0.003854,0.000000,...,0.000000,0.001499,0.000000,0.000000,0.000000,0.016971,0.008168,0.001463,0.000000,0.00000
4,932587626,0.008744,0.000000,0.001715,0.000891,0.000193,0.036889,0.000000,0.001461,0.002363,...,0.000000,0.000000,0.001272,0.002376,0.008026,0.000000,0.000000,0.013404,0.008881,0.00000


In [ ]:
##Run below if want to see topics of any of above
# display_topics(nmf_model, tfidf4.get_feature_names(), 20)

LSA Models:

In [70]:
lsa4_10 = TruncatedSVD(10)
doc_topic_LSA_10 = lsa4_10.fit_transform(bag_of_words_4)

In [78]:
lsa4_10_df = pd.DataFrame(data=doc_topic_LSA_10)
lsa4_10_df.shape

(5902, 10)

In [94]:
df_lsa_model_10 = pd.merge(df_topic_base, lsa4_10_df, left_index=True, right_index=True)
df_lsa_model_10.head()

,id,0,1,2,3,4,5,6,7,8,9
0,269726791,0.118763,0.182434,-0.038079,0.037636,-0.051506,0.044444,-0.008260,0.008037,-0.010564,0.003127
1,1697621882,0.052765,-0.033592,-0.045500,-0.030895,-0.016722,0.016826,0.005462,0.004476,0.061573,-0.006286
2,2046938895,0.017983,0.001345,-0.003667,0.031016,0.002105,-0.021108,0.007550,-0.000639,-0.007482,-0.020165
3,1642293087,0.035800,-0.011461,-0.010412,0.005743,-0.011991,0.018157,0.003074,-0.032434,0.027043,-0.012337
4,932587626,0.085046,-0.055639,-0.039331,0.035681,0.009420,0.001624,-0.025851,-0.004613,0.022707,-0.003880


In [71]:
lsa4_15 = TruncatedSVD(15)
doc_topic_LSA_15 = lsa4_15.fit_transform(bag_of_words_4)

In [79]:
lsa4_15_df = pd.DataFrame(data=doc_topic_LSA_15)
lsa4_15_df.shape

(5902, 15)

In [95]:
df_lsa_model_15 = pd.merge(df_topic_base, lsa4_15_df, left_index=True, right_index=True)
df_lsa_model_15.head()

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,269726791,0.118764,0.182471,-0.037278,0.037991,-0.051005,0.042413,0.003926,-0.011304,0.012133,0.002246,-0.023419,0.014255,0.014478,-0.013392,-0.002236
1,1697621882,0.052766,-0.033623,-0.045656,-0.032438,-0.015115,0.013885,0.013147,-0.007631,-0.035394,-0.022078,0.039715,0.019619,-0.030566,-0.019471,0.011480
2,2046938895,0.017983,0.001346,-0.003049,0.031899,0.002941,-0.021489,0.008210,0.005601,0.008068,-0.016804,-0.001240,-0.000690,-0.009137,-0.008015,-0.013993
3,1642293087,0.035803,-0.011546,-0.010484,0.004713,-0.005824,0.012611,0.007321,0.027702,-0.055404,-0.015021,0.017560,-0.008230,-0.078275,-0.029685,-0.002625
4,932587626,0.085047,-0.055690,-0.040885,0.035537,0.009357,0.012964,-0.037358,-0.018635,-0.019195,-0.004251,-0.033567,0.070771,0.011890,0.049851,0.002826


In [72]:
lsa4_20 = TruncatedSVD(20)
doc_topic_LSA_20 = lsa4_20.fit_transform(bag_of_words_4)

In [80]:
lsa4_20_df = pd.DataFrame(data=doc_topic_LSA_20)
lsa4_20_df.shape

(5902, 20)

In [96]:
df_lsa_model_20 = pd.merge(df_topic_base, lsa4_20_df, left_index=True, right_index=True)
df_lsa_model_20.head()

,id,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,269726791,0.118766,0.182456,-0.037552,0.039314,-0.049405,0.045537,0.004301,-0.002642,0.001682,...,-0.013945,0.002372,0.015489,0.015056,0.003257,0.006292,0.004925,0.000556,0.038885,-0.008544
1,1697621882,0.052764,-0.033601,-0.045470,-0.032412,-0.014509,0.016176,0.016509,0.001259,-0.032337,...,0.015372,-0.001186,-0.056484,0.035841,-0.004553,-0.014576,0.038550,0.009815,-0.044028,0.041492
2,2046938895,0.017983,0.001348,-0.003249,0.031486,0.004197,-0.018840,0.004707,-0.001923,0.007656,...,0.004209,-0.011120,-0.020870,-0.014775,0.008465,0.005519,0.000405,-0.006960,0.003541,0.009025
3,1642293087,0.035802,-0.011551,-0.010231,0.004649,-0.009999,0.012453,0.011625,0.033904,-0.016249,...,-0.014432,0.022894,-0.084028,0.022753,-0.000362,-0.028140,-0.028354,0.009361,0.000377,-0.019683
4,932587626,0.085047,-0.055655,-0.040046,0.035811,0.010295,0.007788,-0.035593,0.015810,-0.026226,...,-0.015908,-0.063662,0.052464,-0.019307,-0.016352,0.022208,0.035765,0.008037,-0.014902,0.022241


In [73]:
lsa4_25 = TruncatedSVD(25)
doc_topic_LSA_25 = lsa4_25.fit_transform(bag_of_words_4)

In [81]:
lsa4_25_df = pd.DataFrame(data=doc_topic_LSA_25)
lsa4_25_df.shape

(5902, 25)

In [97]:
df_lsa_model_25 = pd.merge(df_topic_base, lsa4_25_df, left_index=True, right_index=True)
df_lsa_model_25.head()

,id,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,269726791,0.118766,0.182473,-0.037638,0.039099,-0.049644,-0.043614,0.002513,-0.008042,0.007658,...,0.001354,-0.006548,0.010341,-0.001885,0.010670,-0.023039,0.023898,-0.023569,-0.036885,-0.011361
1,1697621882,0.052765,-0.033601,-0.045561,-0.031533,-0.013853,-0.015841,0.009053,0.003930,-0.031369,...,-0.025432,0.017176,0.020840,-0.033739,-0.009294,0.018991,-0.031492,-0.045181,-0.004206,-0.045321
2,2046938895,0.017983,0.001342,-0.003245,0.031959,0.005199,0.018718,0.008327,0.003250,0.007239,...,0.016408,-0.001985,-0.008515,-0.011488,-0.013514,-0.007318,0.012019,0.031009,0.000805,0.000075
3,1642293087,0.035803,-0.011539,-0.010478,0.004751,-0.008754,-0.012308,0.003959,0.031903,-0.024405,...,-0.008594,-0.027133,-0.024705,-0.021868,-0.002534,-0.048407,-0.001362,-0.032993,-0.008416,0.030770
4,932587626,0.085046,-0.055654,-0.040304,0.034692,0.008419,-0.005710,-0.031724,0.001652,-0.027140,...,0.006802,0.047156,-0.001675,-0.073341,0.031170,-0.068428,-0.050250,0.012359,0.038526,-0.013473


Pickling nmf models:

In [98]:
with open('/Users/robertpagano/metis_data/project_4/text_dataframes/clustering/df_nmf_model_10.pickle', 'wb') as to_write:
    pickle.dump(df_nmf_model_10, to_write)

In [99]:
with open('/Users/robertpagano/metis_data/project_4/text_dataframes/clustering/df_nmf_model_15.pickle', 'wb') as to_write:
    pickle.dump(df_nmf_model_15, to_write)

In [100]:
with open('/Users/robertpagano/metis_data/project_4/text_dataframes/clustering/df_nmf_model_20.pickle', 'wb') as to_write:
    pickle.dump(df_nmf_model_20, to_write)

In [101]:
with open('/Users/robertpagano/metis_data/project_4/text_dataframes/clustering/df_nmf_model_25.pickle', 'wb') as to_write:
    pickle.dump(df_nmf_model_25, to_write)

Pickling lsa models:

In [106]:
with open('/Users/robertpagano/metis_data/project_4/text_dataframes/clustering/df_lsa_model_10.pickle', 'wb') as to_write:
    pickle.dump(df_lsa_model_10, to_write)

In [107]:
with open('/Users/robertpagano/metis_data/project_4/text_dataframes/clustering/df_lsa_model_15.pickle', 'wb') as to_write:
    pickle.dump(df_lsa_model_15, to_write)

In [108]:
with open('/Users/robertpagano/metis_data/project_4/text_dataframes/clustering/df_lsa_model_20.pickle', 'wb') as to_write:
    pickle.dump(df_lsa_model_20, to_write)

In [109]:
with open('/Users/robertpagano/metis_data/project_4/text_dataframes/clustering/df_lsa_model_25.pickle', 'wb') as to_write:
    pickle.dump(df_lsa_model_25, to_write)